In [1]:
# Bibliotheken importieren
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re
import os
from symspellpy import SymSpell, Verbosity
from importlib.resources import files, as_file

In [2]:
# Verzeichnisse für Modelle und Plots erstellen, falls sie nicht existieren
os.makedirs('../models', exist_ok=True)
os.makedirs('../results', exist_ok=True)

In [3]:
# Download für englischensprachigen Datensatz
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------------ --------------------- 5.8/12.8 MB 35.1 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 34.2 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 30.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# Initialisierungen & Downloads
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')
    nlp = spacy.load('en_core_web_sm')
    print("NLP-Bibliotheken erfolgreich initialisiert.")
except Exception as e:
    print("Fehler beim Laden der NLP-Komponenten:")
    print(e)

# Spacy-Modell laden
nlp = spacy.load('en_core_web_sm')

# Prüfen ob der Datensatz vorhanden ist und laden
try: 
    # Pfad ggf. anpassen
    data = pd.read_csv('../data/consumer_complaints.csv', low_memory=False)
    print(f"Datensatz vorhanden und geladen: {data.shape[0]} Zeilen und {data.shape[1]} Spalten")
except FileNotFoundError:
    print("Datei nicht gefunden. Bitte lade den Datensatz herunter und speichere ihn im 'data'-Ordner.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adrig.AG-1337\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adrig.AG-1337\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adrig.AG-1337\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


NLP-Bibliotheken erfolgreich initialisiert.
Datensatz vorhanden und geladen: 555957 Zeilen und 18 Spalten


In [5]:
# SymSpell initialisieren, https://github.com/wolfgarbe/SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Lade die Ressource aus dem symspellpy-Paket
# Pfad zur Wörterbuch-Datei holen
with as_file(files("symspellpy") / "frequency_dictionary_en_82_765.txt") as dictionary_path:
    sym_spell.load_dictionary(str(dictionary_path), term_index=0, count_index=1)


In [6]:
# Zeigt die ersten 5 Zeilen des DataFrames an (Vorschau auf den Datensatz)
data.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [7]:
# Zeigt die Struktur des DataFrames: Spaltennamen, Datentypen und Anzahl der Nicht-Null-Werte
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   date_received                 555957 non-null  object
 1   product                       555957 non-null  object
 2   sub_product                   397635 non-null  object
 3   issue                         555957 non-null  object
 4   sub_issue                     212622 non-null  object
 5   consumer_complaint_narrative  66806 non-null   object
 6   company_public_response       85124 non-null   object
 7   company                       555957 non-null  object
 8   state                         551070 non-null  object
 9   zipcode                       551452 non-null  object
 10  tags                          77959 non-null   object
 11  consumer_consent_provided     123458 non-null  object
 12  submitted_via                 555957 non-null  object
 13 

In [8]:
# Zeigt statistische Kennzahlen für numerische Spalten (z. B. Mittelwert, Standardabweichung, Min/Max)
data.describe()

,complaint_id
count,5.559570e+05
mean,9.600510e+05
std,5.504296e+05
min,1.000000e+00
25%,4.863230e+05
50%,9.737830e+05
75%,1.441702e+06
max,1.895894e+06


In [9]:
# Zählt die Anzahl der fehlenden (NaN) Werte in jeder Spalte
data.isnull().sum()

date_received                        0
product                              0
sub_product                     158322
issue                                0
sub_issue                       343335
consumer_complaint_narrative    489151
company_public_response         470833
company                              0
state                             4887
zipcode                           4505
tags                            477998
consumer_consent_provided       432499
submitted_via                        0
date_sent_to_company                 0
company_response_to_consumer         0
timely_response                      0
consumer_disputed?                   0
complaint_id                         0
dtype: int64

In [10]:
# Prüft, ob die Spalte 'consumer_complaint_narrative' im DataFrame existiert
if 'consumer_complaint_narrative' in data.columns:
    # Filtert nur die Zeilen, in denen ein Beschwerdetext vorhanden ist (ohne NaN)
    complaints = data[data['consumer_complaint_narrative'].notna()]['consumer_complaint_narrative']
    # Gibt die Anzahl der Beschwerden mit Text aus
    print(f"Anzahl der Beschwerden mit Text: {len(complaints):,} von {len(data):,}".replace(",","."))
else:
    print("Spalte mit Beschwerdetexten nicht gefunden. Überprüfe die Spaltennamen:")
    print(data.columns.tolist())

Anzahl der Beschwerden mit Text: 66.806 von 555.957


In [ ]:
# Funktion zur Textvorverarbeitung
def preprocess_text(text):
    # Prüfen ob text ein string ist
    if not isinstance(text, str):
        return ""

    # 1. Kleinbuchstaben
    text = text.lower()

    # 2. Entfernen von Sonderzeichen und Zahlen (nur Buchstaben und Leerzeichen behalten)
    text = re.sub(r'[^a-z\s]', '', text) # Nur Kleinbuchstaben und Leerzeichen beibehalten, wegen ^ nicht diese Zeichen entfernen !!!

    # 3. Tokenisierung (Text in Wörter zerlegen)
    tokens = word_tokenize(text, preserve_line=True)

    # 4. Stoppwörter entfernen (häufige Wörter wie 'the', 'is', 'in')
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatisierung (Wörter auf ihre Grundform zurückführen, z.B. 'running' -> 'run')
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 5.5 Entfernen von Mustern wie 'xx', 'xxx', 'xxxx' usw. | Dieser Schritt ist erst nach der Erstellung der ersten Tabelle mit TF-IDF aufgefallen.
    # re.fullmatch prüft das ganze wort auf das Muster
    tokens = [word for word in tokens if not re.fullmatch(r'x{2,}', word)]

    # 5.6 : Entfernen von Wörtern, die x-Sequenzen enthalten (z.B. "xxxx1234" oder "xxxxbank")
    tokens = [word for word in tokens if not re.search(r'x{2,}', word)]

    # 6. Entfernen von sehr kurzen Wörtern (oft Rauschen)
    tokens = [word for word in tokens if len(word) > 2]

    # 7. Rechtschreibkorrektur mit SymSpellPy für allgemeines Englisch
    corrected_tokens = []
    for word in tokens:
        # Nur für Wörter mit mindestens 3 Buchstaben die Rechtschreibkorrektur anwenden
        if len(word) <= 2:
            corrected_tokens.append(word)
            continue
            
        # SymSpell für allgemeine Rechtschreibkorrektur verwenden
        # https://symspellpy.readthedocs.io/en/latest/api/symspellpy.html
        try:
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
            if suggestions:
                # Wähle die wahrscheinlichste Korrektur, wenn das Konfidenzmaß hoch genug ist
                if suggestions[0].distance <= 1:  # Nur Korrekturen mit geringer Edit-Distanz
                    corrected_tokens.append(suggestions[0].term)
                else:
                    corrected_tokens.append(word)  # Original behalten wenn unsicher
            else:
                corrected_tokens.append(word)
        except:
            corrected_tokens.append(word)

    # 8. Entfernen von sehr kurzen Wörtern (oft Rauschen)
    corrected_tokens = [word for word in corrected_tokens if len(word) > 2]

    # Rückgabe als einzelner String mit Leerzeichen getrennt
    return ' '.join(corrected_tokens)

In [12]:
# Vorverarbeitung auf einer kleinen Stichprobe testen
print("\nTest der Vorverarbeitung an Beispielen:")
sample = complaints.sample(5)
for i, text in enumerate(sample):
    print(f"Original {i+1}: {text[:100]}...")
    print(f"Verarbeitet {i+1}: {preprocess_text(text)[:100]}...")
    print("-" * 80)


Test der Vorverarbeitung an Beispielen:
Original 1: A request was made through customer service to stop an online payment scheduled from Chase Bank. 
Th...
Verarbeitet 1: request made customer service stop online payment scheduled chase bank payment stopped amount credit...
--------------------------------------------------------------------------------
Original 2: I have contacted Ocwen several times in the past year regarding mortgage assistance. We have a ballo...
Verarbeitet 2: contacted owen several time past year regarding mortgage assistance balloon payment due unsuccessful...
--------------------------------------------------------------------------------
Original 3: Greetings. I am writing because in attempting to settle a cellular phone debt with Enhanced Recovery...
Verarbeitet 3: greeting writing attempting settle cellular phone debt enhanced recovery corporation credit card deb...
--------------------------------------------------------------------------------
Original 4:

In [13]:
# Vorverarbeitung auf alle Texte anwenden (kann etwas dauern | bei mir ca. 3 Minuten))
# Hinweis: Bei sehr großen Datensätzen kann dies länger dauern.
processed_complaints = complaints.apply(preprocess_text)

# Speichern für die nächste Phase
processed_df = pd.DataFrame({'processed_text': processed_complaints})
processed_df.to_csv('../data/processed_complaints_v2.csv', index=False)